In [1]:
import openml

In [5]:
import torch
import numpy as np
from tabpfn_extensions import TabPFNClassifier, TabPFNRegressor
from tabpfn_extensions.embedding import TabPFNEmbedding
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

# Load data
X, y = load_wine(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
suite = openml.study.get_suite(99)
print(suite)
tasks = suite.tasks

OpenML Benchmark Suite
ID..............: 99
Name............: OpenML-CC18 Curated Classification benchmark
Status..........: active
Main Entity Type: task
Study URL.......: https://www.openml.org/s/99
# of Data.......: 72
# of Tasks......: 72
Creator.........: https://www.openml.org/u/1
Upload Time.....: 2019-02-21 18:47:13


In [7]:
clf = TabPFNClassifier(n_estimators=1, device='cuda:1' if torch.cuda.is_available() else 'cpu', ignore_pretraining_limits=True)
embedding_extractor = TabPFNEmbedding(tabpfn_clf=clf, n_fold=0)

import tqdm

train_dict = {}
val_dict = {}
test_dict = {}

train_labels = {}
val_labels = {}
test_labels = {}

for task_id in tqdm.tqdm(tasks):
    task = openml.tasks.get_task(task_id)
    X, y = task.get_X_and_y()

    if len(X) > 5000:
        perm = np.random.permutation(len(X))
        X = X[perm[:5000]]
        y = y[perm[:5000]]

    if len(np.unique(y)) > 10:
        continue

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    train_embeddings = embedding_extractor.get_embeddings(
        X_train, # np.random.normal(size=(X_train.shape)),
        y_train, # np.zeros_like(y_train),
        X_train,
        data_source='test',
    )[0]
    val_embeddings = embedding_extractor.get_embeddings(
        X_train, # np.random.normal(size=(X_train.shape)),
        y_train, # np.zeros_like(y_train),
        X_val,
        data_source='test',
    )[0]
    test_embeddings = embedding_extractor.get_embeddings(
        X_train, # np.random.normal(size=(X_train.shape)),
        y_train, # np.zeros_like(y_train),
        X_test,
        data_source='test',
    )[0]

    train_dict[task_id] = train_embeddings
    val_dict[task_id] = val_embeddings
    test_dict[task_id] = test_embeddings

    train_labels[task_id] = y_train
    val_labels[task_id] = y_val
    test_labels[task_id] = y_test



  0%|          | 0/72 [00:00<?, ?it/s]/tmp/ipykernel_508015/3714553198.py:16: FutureWarning: Support for `dataset_format='array'` will be removed in 0.15,start using `dataset_format='dataframe' to ensure your code will continue to work. You can use the dataframe's `to_numpy` function to continue using numpy arrays.
  X, y = task.get_X_and_y()
/home/malonso/miniconda3/envs/mantisenv/lib/python3.10/site-packages/openml/tasks/task.py:334: FutureWarning: Support for `dataset_format='array'` will be removed in 0.15,start using `dataset_format='dataframe' to ensure your code will continue to work. You can use the dataframe's `to_numpy` function to continue using numpy arrays.
  X, y, _, _ = dataset.get_data(
  1%|▏         | 1/72 [00:02<02:35,  2.18s/it]/tmp/ipykernel_508015/3714553198.py:16: FutureWarning: Support for `dataset_format='array'` will be removed in 0.15,start using `dataset_format='dataframe' to ensure your code will continue to work. You can use the dataframe's `to_numpy` func

In [ ]:
import json
with open('tabPFN/train_embeddings.json', 'w') as f:
    json.dump({k: v.tolist() for k, v in train_dict.items()}, f)
with open('tabPFN/val_embeddings.json', 'w') as f:
    json.dump(val_dict, f)
with open('tabPFN/test_embeddings.json', 'w') as f:
    json.dump(test_dict, f)
with open('tabPFN/train_labels.json', 'w') as f:
    json.dump(train_labels, f)
with open('tabPFN/val_labels.json', 'w') as f:
    json.dump(val_labels, f)
with open('tabPFN/test_labels.json', 'w') as f:
    json.dump(test_labels, f)

TypeError: Object of type ndarray is not JSON serializable

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Train and evaluate TabPFN embeddings (vanilla)


embedding_extractor = TabPFNEmbedding(tabpfn_clf=clf, n_fold=0)
train_embeddings = embedding_extractor.get_embeddings(
    X_train, # np.random.normal(size=(X_train.shape)),
    y_train, # np.zeros_like(y_train),
    X_train,
    data_source='test',

)
test_embeddings = embedding_extractor.get_embeddings(
    X_train, # np.random.normal(size=(X_train.shape)),
    y_train, # np.zeros_like(y_train),
    X_test,
    data_source='test',
)

model = LogisticRegression()
model.fit(train_embeddings[0], y_train)
y_pred = model.predict(test_embeddings[0])
print(
    f"Logistic Regression with TabPFN (Vanilla) Accuracy: {accuracy_score(y_test, y_pred):.4f}",
)

Logistic Regression with TabPFN (Vanilla) Accuracy: 0.9781


In [ ]:
train_embeddings.shape

(1, 2556, 192)

In [2]:
import torch
import json
with open('tabPFN/train_embeddings.json', 'r') as f:
    train_dict = json.load(f)
    train_dict = {k: torch.tensor(v).float() for k, v in train_dict.items()}
with open('tabPFN/val_embeddings.json', 'r') as f:
    val_dict = json.load(f)
    val_dict = {k: torch.tensor(v).float() for k, v in val_dict.items()}
with open('tabPFN/test_embeddings.json', 'r') as f:
    test_dict = json.load(f)
    test_dict = {k: torch.tensor(v).float() for k, v in test_dict.items()}
with open('tabPFN/train_labels.json', 'r') as f:
    train_labels = json.load(f)
    train_labels = {k: torch.tensor(v).long() for k, v in train_labels.items()}
with open('tabPFN/val_labels.json', 'r') as f:
    val_labels = json.load(f)
    val_labels = {k: torch.tensor(v).long() for k, v in val_labels.items()}
with open('tabPFN/test_labels.json', 'r') as f:
    test_labels = json.load(f)
    test_labels = {k: torch.tensor(v).long() for k, v in test_labels.items()}

In [ ]:
for task_id in train_dict.keys():
    print(task_id, '-', train_dict[task_id].shape[0], val_dict[task_id].shape[0], test_dict[task_id].shape[0], test_dict[task_id]

3 - 1278 320 1598
11 - 249 63 313
12 - 800 200 1000
14 - 800 200 1000
15 - 279 70 350
16 - 800 200 1000
18 - 800 200 1000
22 - 800 200 1000
23 - 588 148 737
28 - 2000 500 2500
29 - 276 69 345
31 - 400 100 500
32 - 2000 500 2500
37 - 307 77 384
43 - 1840 460 2301
45 - 1276 319 1595
49 - 383 96 479
53 - 338 85 423
219 - 2000 500 2500
2074 - 2000 500 2500
2079 - 294 74 368
3021 - 1508 378 1886
3549 - 336 84 421
3560 - 318 80 399
3573 - 2000 500 2500
3902 - 583 146 729
3903 - 624 157 782
3904 - 2000 500 2500
3913 - 208 53 261
3917 - 843 211 1055
3918 - 443 111 555
7592 - 2000 500 2500
9910 - 1500 375 1876
9946 - 227 57 285
9952 - 2000 500 2500
9957 - 421 106 528
9960 - 2000 500 2500
9964 - 636 160 797
9971 - 232 59 292
9976 - 1040 260 1300
9977 - 2000 500 2500
9978 - 1013 254 1267
9981 - 432 108 540
9985 - 2000 500 2500
10093 - 548 138 686
10101 - 299 75 374
14952 - 2000 500 2500
14954 - 216 54 270
14965 - 2000 500 2500
14969 - 2000 500 2500
14970 - 2000 500 2500
125920 - 200 50 250
146195